# Experimenting with the simulated data

In [9]:
from transit_system import TransitSystem
import numpy as np
import pandas as pd
import pickle as pkl
import json
import networkx as nx

In [5]:
with open("transit_system_config.json", "r") as file:
    transit_system_config = json.load(file)

transit_system = TransitSystem(**transit_system_config, seed=97)
graph = transit_system.topology.topology.copy()

In [8]:
graph.nodes(data=True)

NodeDataView({0: {}, 1: {}, 3: {}, 4: {}, 5: {}, 6: {}, 7: {}, 8: {}, 9: {}, 10: {}, 11: {}, 12: {}, 13: {}, 14: {}, 15: {}, 16: {}, 17: {}, 18: {}, 19: {}, 20: {}, 21: {}, 22: {}, 23: {}, 24: {}, 25: {}, 26: {}, 27: {}, 28: {}, 29: {}, 30: {}, 31: {}, 32: {}, 33: {}, 34: {}, 35: {}, 36: {}, 37: {}, 38: {}, 39: {}, 40: {}, 41: {}, 42: {}, 43: {}, 44: {}, 46: {}, 47: {}, 48: {}, 50: {}, 51: {}, 52: {}, 53: {}, 54: {}, 55: {}, 56: {}, 57: {}, 58: {}, 59: {}, 60: {}, 61: {}, 62: {}, 63: {}, 64: {}, 65: {}, 66: {}, 67: {}, 68: {}, 69: {}, 71: {}, 72: {}, 73: {}, 74: {}, 75: {}, 76: {}, 77: {}, 78: {}, 79: {}, 80: {}, 81: {}, 82: {}, 83: {}, 84: {}, 85: {}, 86: {}, 87: {}, 88: {}, 89: {}, 91: {}, 92: {}, 93: {}, 94: {}, 95: {}, 96: {}, 97: {}, 98: {}, 99: {}, 100: {}, 101: {}, 102: {}, 103: {}, 104: {}, 105: {}, 106: {}, 107: {}, 108: {}, 109: {}, 110: {}, 111: {}, 112: {}, 114: {}, 115: {}, 117: {}, 118: {}, 119: {}, 120: {}, 121: {}})

In [ ]:
# seeds = [7594897, 8032337, 3297648, 438859, 8685788, 9143448]
seeds = [7594897]

datas = []
analysis_interval_sec = 30
analysis_period_days = 1
hours_of_opperation_per_day = 18

for num_busses_per_route in [1]:
    for seed in seeds:
        transit_system = TransitSystem(
            analysis_period_sec=analysis_interval_sec,
            num_busses_per_route=num_busses_per_route,
            hours_of_opperation_per_day=hours_of_opperation_per_day,
            min_bus_capacity=100,
            max_bus_capacity=200,
            seed=seed,
        )

        for i in range(0, hours_of_opperation_per_day * 3600, analysis_interval_sec):
            for j in range(analysis_period_days):
                transit_system.step(i)

    datas.append(pd.read_csv("logs/passenger_logs.csv", low_memory=False))

with open("data.pkl", "wb") as file:
    pkl.dump(datas, file)

In [3]:
x = pd.read_csv("logs/passenger_logs.csv", low_memory=False)
x

,seed,time,origin,destination,num_transfers,transfers,waiting_time,travel_time,stranding_counts,distance_traversed,num_stations_traversed,average_travel_speed,total_time_taken
0,7594897,840,287,284,0,NaN,0,450,0,8590.0,2,19.088889,450
1,7594897,840,269,216,0,NaN,270,210,0,4557.0,2,21.700000,480
2,7594897,840,269,216,0,NaN,60,210,0,4557.0,2,21.700000,270
3,7594897,930,279,271,0,NaN,240,90,0,2581.0,2,28.677778,330
4,7594897,1110,269,216,0,NaN,0,300,0,4557.0,2,15.190000,300
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35762,7594897,10020,38,152,0,NaN,5040,390,5,6963.0,7,17.853846,5430
35763,7594897,10020,38,152,0,NaN,4920,390,5,6963.0,7,17.853846,5310
35764,7594897,10020,273,109,0,NaN,2520,540,2,9831.0,9,18.205556,3060
35765,7594897,10020,226,109,0,NaN,660,450,0,7908.0,7,17.573333,1110


In [ ]:
# took 70 mins to run
with open("data.pkl", "rb") as file:
    data = pkl.load(file)

In [ ]:
for i in range(len(data)):
    print(f"number of passengres served by {[1, 5, 10, 20][i]} buses per route per direction", data[i].shape[0]//5000, "thousand")
    display(data[i].groupby("num_transfers")[
    [
        "waiting_time",
        "travel_time",
        "distance_traversed",
        "total_time_taken",
        "num_stations_traversed",
        "stranding_counts"
    ]
    ].median())